# Concurrency Mode

Bridgic runs primarily on an asynchronous event loop, while seamlessly supporting I/O-bound tasks through threads. This design ensures high concurrency across diverse workloads.

## Web content analysis assistant

To explore Bridgic’s support for concurrency, let’s build a web content analysis assistant to summarize and introduce the main content of a given web page. The steps are as follows:

1. Crawl relevant content of the input url.
2. Summarize and introduce main content

### 1. Crawl relevant content

Taking the *[Books to Scrape](http://books.toscrape.com/index.html)* website as an example, we are given the url of a book page on the website. Like this: 

In [ ]:
url="http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"

The page looks like this:

<br>
<div style="text-align: center;">
<img src="../../../imgs/books_to-scrape.png" alt="Parameter Passing" width="800" height="600">
</div>
<br>

> Note: We use [Books to Scrape](http://books.toscrape.com/index.html), a demo website created specifically for practicing web scraping, to introduce in this tutorial. Please note that the purpose of writing a crawler here is **not** to build a real scraper, but to provide a simple and safe example to demonstrate how Bridgic handles both synchronous and asynchronous execution models.

We use `requests` to obtain the web content of the given url. Use `pip install requests` to install `requests` package and crawl the page like this:

In [ ]:
import requests

def get_web_content(url):  # will return the web content of the given url
    response = requests.get(url)
    return response.text

### 2. Summarize and introduce main content

We create an agent, input a url and crawl the corresponding page, and then let the model summarize the main content of the web page.

Initialize the runtime environment.

In [ ]:
import os

# Set the API base and key.
_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

# Import the necessary modules.
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.model.types import Message, Role
from bridgic.llms.openai_like import OpenAILikeLlm

llm = OpenAILikeLlm(api_base=_api_base, api_key=_api_key, timeout=30)

Let's write web content analysis assistant.

In [ ]:
class WebContentAnalysisAgent(GraphAutoma):
    @worker(is_start=True)
    def crawl_web_content(self, url: str) -> str:
        response = requests.get(url)
        return response.text

    @worker(dependencies=["crawl_web_content"], is_output=True)
    async def analyze_web_content(self, content: str) -> str:
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="You are a web content analysis assistant. Your task is to analyze the given web content and summarize the main content.", role=Role.SYSTEM),
                Message.from_text(text=content, role=Role.USER),
            ]
        )
        return response.message.content

Now, let's use it to help us analyze the content.

In [22]:
web_content_analysis_agent = WebContentAnalysisAgent()

# Input the url of the web page to be analyzed.
url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"

# Call the agent to analyze the web content.
res = await web_content_analysis_agent.arun(url)

# Print the result.
print(f'- - - - - result - - - - -')
print(res)
print(f'- - - - - end - - - - -')


- - - - - result - - - - -
The provided HTML content is from a product page on **Books to Scrape**, a demo website designed for web scraping education. Here's a clear summary of the main content:

---

### **Main Content Summary: A Light in the Attic**

- **Product Title**: *A Light in the Attic*  
- **Author**: Shel Silverstein  
- **Category**: Poetry  
- **Product Type**: Book (Poetry with illustrations)  
- **Price**: £51.77 (excl. and incl. tax; tax is £0.00)  
- **Availability**: In stock (22 units available)  
- **Rating**: 5 stars (all full stars)  
- **Number of Reviews**: 0  

---

### **Product Description Highlights**
- Celebrates its 20th anniversary with a special edition.
- Known for humorous, creative, and rhythmic poetry that appeals to both children and adults.
- Features classic verses such as *"Rockabye Baby"*:
  > *"Rockabye baby, in the treetop / Don't you know a treetop / Is no safe place to rock?"*
- Described as a timeless classic that brings joy and laughter t

<div style="text-align: center; margin: 2rem 0;">
<hr style="border: none; border-top: 2px solid #e2e8f0;">
</div>

## What have we learnt?

It can be seen from the example that Bridgic can seamlessly schedule both asynchronous and synchronous workers within the same automa. Although `crawl_web_content` performs a blocking network request, Bridgic automatically dispatches it to a thread so that the event loop remains unblocked. Meanwhile, `analyze_web_content` runs asynchronously within the event loop. Refer to [`Worker`](../../../../reference/bridgic-core/bridgic/core/automa/worker/#bridgic.core.automa.worker.Worker) for details.

In this way, Bridgic keeps an asynchronous-first design, but also provides built-in support for I/O-bound operations through its thread pool, ensuring smooth execution across different types of workloads.